In [1]:
import ImportData
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
df = ImportData.importDataAsTable(False, False)

In [3]:
df.head()

,IC50,cell_line,drug_id
0,5.457453,ES3,176870
1,6.177945,ES5,176870
2,3.954488,EW-11,176870
3,5.429563,NCI-H1395,176870
4,6.239001,NCI-H1770,176870


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2)
y_test = test_df["IC50"]

# Based on the cell_line average

In [5]:
train_cell_line_mean = train_df.groupby(['cell_line'])['IC50'].mean()

In [6]:
y_pred_cell_line = [train_cell_line_mean[cell_line] for cell_line in test_df["cell_line"]]

In [7]:
mean_squared_error(y_test, y_pred_cell_line)

10.00828641040131

# Based on the drug_id average

In [8]:
train_drug_id_mean = train_df.groupby(['drug_id'])['IC50'].mean()

In [9]:
y_pred_drug_id = [train_drug_id_mean[drug_id] for drug_id in test_df["drug_id"]]

In [10]:
mean_squared_error(y_test, y_pred_drug_id)

3.5168309182395943

# Based on the drug_id average and cell_line average

In [11]:
train_df.loc[:,"drug_id_average"] = [train_drug_id_mean[drug_id] for drug_id in train_df["drug_id"]]
train_df.loc[:,"cell_line_average"] = [train_cell_line_mean[cell_line] for cell_line in train_df["cell_line"]]

test_df.loc[:,"drug_id_average"] = [train_drug_id_mean[drug_id] for drug_id in test_df["drug_id"]]
test_df.loc[:,"cell_line_average"] = [train_cell_line_mean[cell_line] for cell_line in test_df["cell_line"]]

/home/soeren/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/soeren/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
X_train = train_df[['drug_id_average', 'cell_line_average']]
y_train = train_df['IC50']

In [13]:
reg = LinearRegression().fit(X_train, y_train)

In [14]:
y_pred = reg.predict(test_df[['drug_id_average', 'cell_line_average']])

In [15]:
mean_squared_error(y_test, y_pred)

3.0117750379297905